In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jan 13 07:52:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Cloning DarkNet

In [2]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14665, done.
remote: Total 14665 (delta 0), reused 0 (delta 0), pack-reused 14665
Receiving objects: 100% (14665/14665), 13.25 MiB | 7.72 MiB/s, done.
Resolving deltas: 100% (9977/9977), done.


In [3]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

CONFIGURING YOLOV3 file

In [5]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

.names and .data file creation

In [7]:
!echo -e 'Mask\nNo Mask' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [8]:
!cp cfg/yolov3_training.cfg /content/drive/MyDrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /content/drive/MyDrive/yolov3/classes.txt

In [9]:
!mkdir data/obj

In [10]:
!unzip /content/drive/MyDrive/yolov3/images.zip -d data/obj

Archive:  /content/drive/MyDrive/yolov3/images.zip
  inflating: data/obj/images/_104224787_tv050424063.jpg  
  inflating: data/obj/images/_104224787_tv050424063.txt  
  inflating: data/obj/images/_112342926_airportfacemask.jpg  
  inflating: data/obj/images/_112342926_airportfacemask.txt  
  inflating: data/obj/images/_113397291_face_coverings.png  
  inflating: data/obj/images/_113397291_face_coverings.txt  
  inflating: data/obj/images/_115265768_how_not_to_wear_a_mask5_640-nc.png  
  inflating: data/obj/images/_115265768_how_not_to_wear_a_mask5_640-nc.txt  
  inflating: data/obj/images/00well-masks-nose-mobileMasterAt3x-v2.png  
  inflating: data/obj/images/00well-masks-nose-mobileMasterAt3x-v2.txt  
  inflating: data/obj/images/00well-mask-too-low-mobileMasterAt3x-v2.png  
  inflating: data/obj/images/00well-mask-too-low-mobileMasterAt3x-v2.txt  
  inflating: data/obj/images/04082020_3M-Mask_094031-780x498.jpg  
  inflating: data/obj/images/04082020_3M-Mask_094031-780x498.txt  
  i

In [11]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [12]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-01-13 07:56:02--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   448KB/s    in 6m 9s   

2021-01-13 08:02:12 (430 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/yolov3/yolov3_training_last.weights -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x